<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/11_1_22_gossipcop_dgl_GCN_text_embs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip -qq install jsonlines

In [8]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [9]:
from pathlib import Path
base_dir = Path("/gdrive/MyDrive/ResearchFND")
assert base_dir.exists()

## Data

In [10]:
import pandas as pd
import ast
import json

In [11]:
dataset_id = 'gossipcop'
text_embeddings = 'sbert' # options: sbert, ft

In [12]:
df = pd.read_csv(base_dir/f'{dataset_id}_agg.csv')
df.head(2)

,title,text,tweets,retweets,label,url,num_retweets,log_num_retweets,num_tweets,log_num_tweets
0,Kendall Kylie Jenner Jenner NOT Upset Up...,NaN,[],"['995423424741888001', '995461685166202880', '...",fake,NaN,3,1.386294,0,0.0
1,Kim Kardashian Dethroned Dethroned By Khlo...,NaN,[],"['848843565027516416', '849030801970868224', '...",fake,NaN,3,1.386294,0,0.0


In [13]:
df['tweets'] = df.tweets.map(ast.literal_eval)

In [14]:
users_tweeted = df.tweets.map(lambda x: [int(e['user_id']) for e in x])

In [15]:
len(users_tweeted), sum(users_tweeted.map(len) > 0)

(19968, 2117)

## GNN

### Data

In [16]:
#%%capture
!pip install dgl wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 5.2 MB/s 
     |████████████████████████████████| 1.9 MB 64.5 MB/s 
     |████████████████████████████████| 291 kB 62.7 MB/s 
     |████████████████████████████████| 182 kB 47.5 MB/s 
     |████████████████████████████████| 166 kB 51.9 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 166 kB 69.1 MB/s 
     |████████████████████████████████| 162 kB 58.2 MB/s 
     |████████████████████████████████| 162 kB 45.7 MB/s 
     |████████████████████████████████| 158 kB 57.7 MB/s 
     |████████████████████████████████| 157 kB 59.2 MB/s 
     |████████████████████████████████| 157 kB 47.4 MB/s 
     |████████████████████████████████| 157 kB 50.3 MB/s 
     |████████████████████████████████| 157 kB 46.2 MB/s 
     |████████████████████████████████| 157 kB 67.7 MB/s 
     |████████████████████████████

In [17]:
%env DGLBACKEND=pytorch

env: DGLBACKEND=pytorch


In [18]:
import os
import json
import jsonlines
import numpy as np
import torch
import dgl

import wandb
import IPython.display as ipd

In [19]:
u2i = {}

follow_src = []
follow_dst = []
with jsonlines.open(base_dir/"followers.jsonl") as reader:
    for line in reader:
        v = line["user_id"]
        if v not in u2i:
            u2i[v] = len(u2i)
        for u in line["followers"]:
            if u not in u2i:
                u2i[u] = len(u2i)
            follow_src.append(u2i[u])
            follow_dst.append(u2i[v])

In [20]:
with jsonlines.open(base_dir/"following.jsonl") as reader:
    for line in reader:
        u = line["user_id"]
        if u not in u2i:
            u2i[u] = len(u2i)
        for v in line["following"]:
            if v not in u2i:
                u2i[v] = len(u2i)
            follow_src.append(u2i[u])
            follow_dst.append(u2i[v])

In [21]:
tweet_src = []
tweet_dst = []

for v, l in users_tweeted.iteritems():
    if not len(l):
        continue
    for u in l:
        u = int(u)
        if u in u2i:
            tweet_src.append(u2i[u])
            tweet_dst.append(v)

In [22]:
text_embs = np.load(base_dir/f'{dataset_id}_{text_embeddings}_fulltext_embeddings.npy')
text_embs.shape

(19968, 768)

In [23]:
num_users = len(set(follow_src+follow_dst+tweet_src))

In [24]:
follow_src = torch.tensor(follow_src)
follow_dst = torch.tensor(follow_dst)
tweet_src = torch.tensor(tweet_src)
tweet_dst = torch.tensor(tweet_dst)

graph = dgl.heterograph({
    ('user', 'follow', 'user'): (follow_src, follow_dst),
    ('user', 'followed-by', 'user'): (follow_dst, follow_src),
    ('user', 'tweet', 'article'): (tweet_src, tweet_dst),
    ('article', 'tweeted-by', 'user'): (tweet_dst, tweet_src)},
    {'article':len(df), 'user':num_users}
)

graph.nodes['user'].data['feat'] = torch.arange(graph.num_nodes('user'))
graph.nodes['article'].data['feat'] = torch.tensor(text_embs)
graph.nodes['article'].data['label'] = torch.tensor((df.label=="real").to_numpy()).long()

In [25]:
graph

Graph(num_nodes={'article': 19968, 'user': 31792},
      num_edges={('article', 'tweeted-by', 'user'): 588, ('user', 'follow', 'user'): 48409, ('user', 'followed-by', 'user'): 48409, ('user', 'tweet', 'article'): 588},
      metagraph=[('article', 'user', 'tweeted-by'), ('user', 'user', 'follow'), ('user', 'user', 'followed-by'), ('user', 'article', 'tweet')])

In [26]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(shuffle=True, random_state=124)

In [27]:
labels = graph.ndata['label']['article']

train_idx, valid_idx = next(skf.split(labels, labels))

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [29]:
n_layers = 3
n_neighbors = 10

In [30]:
sampler = dgl.dataloading.NeighborSampler([n_neighbors]*n_layers)
train_loader = dgl.dataloading.DataLoader(
    graph,
    {'article':train_idx},
    sampler,
    device=device,
    batch_size=64,
    shuffle=True,
    drop_last=False,
    num_workers=0
)

In [31]:
eval_sampler = dgl.dataloading.NeighborSampler([-1]*n_layers)
eval_loader = dgl.dataloading.DataLoader(
    graph,
    {'article':valid_idx},
    eval_sampler,
    device=device,
    batch_size=64,
    shuffle=False,
    drop_last=False,
    num_workers=0
)

In [32]:
batch = next(iter(train_loader))

/usr/local/lib/python3.7/dist-packages/dgl/dataloading/dataloader.py:859: DGLWarning: Dataloader CPU affinity opt is not enabled, consider switching it on (see enable_cpu_affinity() or CPU best practices for DGL [https://docs.dgl.ai/tutorials/cpu/cpu_best_practises.html])
  dgl_warning(f'Dataloader CPU affinity opt is not enabled, consider switching it on '


### Model

In [33]:
from collections import defaultdict
import torch.nn as nn
import torch.nn.functional as F

d_emb_dict = defaultdict(lambda: 64)

def flatten_dict(d):
    for k, v in d.items():
        d[k] = v.flatten(1)
    return d

In [34]:
class NodeEmbedding(nn.Module):
    """
    Node embedding layer. Handles input node features of varying dimentionality.
    
    Parameters:
    - n_nodes: dict[str,int] - a dictionary containing number of nodes per type
    for id only nodes (without precmputed features)
    - d_in: dict[str,int] - a dictionary mapping node type to the node feature dim
    for nodes with precomputed vector features
    - d_emb: int - size of output feature vector for all node types
    - proj_nodes: list[str] - optional list of node types with vector features to be
    processed with linear projection. If None keys of `d_in` are used
    - embed_nodes: list[str] - optional list of node types without vector features to be
    processed with embedding layer. If None keys of `n_nnodes` are used

    Inputs:
    - nx - dict[str,Tensor] - dictionary containing input node features per node type

    Outputs:
    - out - dict[str, Tensor] - dictionary of node embedding tensors of shape [bs, d_emb]
    """
    def __init__(self, n_nodes:dict, d_in:dict, d_emb:int, proj_nodes:list=None, embed_nodes:list=None):
        super().__init__()
        self.proj_nodes = proj_nodes if proj_nodes is not None else list(d_in.keys())
        self.embed_nodes = embed_nodes if embed_nodes is not None else list(n_nodes.keys())
        self.emb = nn.ModuleDict({k:nn.Embedding(n_nodes[k], d_emb) for k in self.embed_nodes})
        self.proj = nn.ModuleDict({k:nn.Linear(d_in[k], d_emb, bias=False) for k in self.proj_nodes})
        self.init()

    def forward(self, nx):
        out = {}
        for k, m  in self.emb.items():
            out[k] = m(nx[k])
        for k, m  in self.proj.items():
            out[k] = m(nx[k])
        return out

    def init(self):
        for _, m in self.emb.items():
            torch.nn.init.xavier_uniform_(m.weight)
        for _, m in self.proj.items():
            torch.nn.init.xavier_uniform_(m.weight)

In [35]:
class Residual(nn.Module):
    """
    Residual connection. Computes output node features as:
    x_out_dst = GraphConv(graph, x_in_all) + x_in_dst
    """
    def __init__(self, conv):
        super().__init__()
        self.conv = conv

    def forward(self, graph, x):
        h = self.conv(graph, x)
        res = x[1]
        return h + res

In [36]:
in_proj = NodeEmbedding({k:graph.num_nodes(k) for k in ["user"]}, {"article":text_embs.shape[1]}, 64)
conv = dgl.nn.HeteroGraphConv({rel:Residual(dgl.nn.GraphConv(64, 64, allow_zero_in_degree=True)) for rel in graph.etypes})

In [37]:
blocks = batch[-1]
block = blocks[0]
x = block.ndata['feat']

with torch.no_grad():
    h = in_proj(x)
    res = conv(block, h)

In [38]:
res['article'].shape, res['user'].shape

(torch.Size([74, 64]), torch.Size([4, 64]))

In [39]:
class Encoder(torch.nn.Module):
    """
    Encoder for heterogenous graph using GCN layer.
    """
    def __init__(self, d_in, d_h, etypes, n_layers=2, dropout=0.0):
        super().__init__()
        ds = [d_in] + [d_h] * n_layers
        self.layers = nn.ModuleList([
            dgl.nn.HeteroGraphConv({
                rel : Residual(dgl.nn.GraphConv(ds[i], ds[i+1], allow_zero_in_degree=True)) for rel in etypes
            }) for i in range(n_layers)
        ])

    def forward(self, blocks, x):
        
        for layer, block in zip(self.layers, blocks):
            x = layer(block, x)
        return x

In [40]:
graph.ndata['feat']['article'].shape

torch.Size([19968, 768])

In [41]:
class GNN(nn.Module):
    
    def __init__(self, g, d_h:int, n_layers:int, tgt_ntype:str, emb_nodes:list=['user'], proj_nodes:list=['article']):
        super().__init__()
        self.tgt_ntype = tgt_ntype
        self.in_proj = NodeEmbedding(
            {k:g.num_nodes(k) for k in emb_nodes}, 
            {k:graph.ndata['feat'][k].shape[1] for k in proj_nodes},
            d_h
        )
        self.encoder = Encoder(d_h, d_h, g.etypes)
        self.head = nn.Linear(d_h, 2)

    def forward(self, blocks, x):
        h = self.in_proj(x)
        h = self.encoder(blocks, h)
        return self.head(h[self.tgt_ntype])

    @torch.no_grad()
    def get_embeddings(self, graph, x):
        h = self.emb(x)
        h = self.encoder(graph, h)
        return h[self.tgt_ntype]

In [42]:
model = GNN(graph, 128, n_layers, 'article')

with torch.no_grad():
    logits = model(blocks, x)

In [43]:
logits.shape

torch.Size([64, 2])

In [44]:
def accuracy(logits, labels):
    return (logits.argmax(-1) == labels).float().mean()

In [45]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(shuffle=True, random_state=124)

In [46]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

metrics = [accuracy_score, f1_score, precision_score, recall_score]
def get_name(score_func):
    return score_func.__name__.split("_")[0]

In [47]:
class AverageMeter:

    def __init__(self, store_vals=False, store_avgs=False):
        self.store_vals = store_vals
        self.store_avgs = store_avgs
        if store_vals: self.values = []
        if store_avgs: self.avgs = []
        self.tot, self.n = 0, 0

    def update(self, v, n=1):
        if self.store_vals: self.values.append(v)
        self.n += n
        self.tot += v*n

    @property
    def avg(self):
        if self.n == 0:
            return
        return self.tot / self.n

    def reset(self):
        if self.store_avgs and self.avg: self.avgs.append(self.avg)
        self.tot, self.n = 0, 0

In [48]:
class_weights = torch.tensor([1/(labels==0).sum(), 1/(labels==1).sum()])*len(labels)

In [49]:
def train(fold, train_idx, valid_idx, params):

    model = GNN(graph, 128, n_layers, 'article')
    opt = torch.optim.Adam(model.parameters(), params['lr'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        opt, factor=params["lr_sched_factor"], patience=params["patience"], verbose=True
    )

    train_loss = AverageMeter(store_avgs=True)
    train_acc = AverageMeter(store_avgs=True)
    valid_loss = AverageMeter(store_avgs=True)
    valid_acc = AverageMeter(store_avgs=True)

    best_acc = 0
    for epoch in range(params['n_epochs']):
        model.train()
        for batch in train_loader:
            blocks = batch[-1]
            x = blocks[0].ndata['feat']
            logits = model(blocks, x)
            
            labels = blocks[-1].dstdata['label']['article']
            loss = F.cross_entropy(logits, labels, weight=class_weights)
            acc = accuracy(logits, labels)
            opt.zero_grad()
            loss.backward()
            opt.step()

            train_loss.update(loss.item(), len(labels))
            train_acc.update(acc, len(labels))

        model.eval()
        for i, batch in enumerate(eval_loader):
            blocks = batch[-1]
            x = blocks[0].ndata['feat']
            with torch.no_grad():
                logits = model(blocks, x)

                labels = blocks[-1].dstdata['label']['article']
                val_loss = F.cross_entropy(logits, labels, weight=class_weights)
                val_acc = accuracy(logits, labels)

                valid_loss.update(val_loss.item(), len(labels))
                valid_acc.update(val_acc, len(labels))
        
        scheduler.step(valid_loss.avg)
        wandb.log({'train_loss':loss.item(), 'train_acc':acc, 'valid_loss':val_loss.item(), 'valid_acc':val_acc}, step=epoch)
        print(f"{epoch+1:>3}: Train loss {train_loss.avg:.4f}, acc {train_acc.avg:.4f}%; validation loss {valid_loss.avg:.4f}, acc {valid_acc.avg:.4f}%")
        
        if valid_acc.avg >= best_acc:
            best_acc = valid_acc.avg
            torch.save(model.state_dict(), f'models/model-{fold}.pt')
        
        train_loss.reset()
        train_acc.reset()
        valid_loss.reset()
        valid_acc.reset()

        

    # load best model and evaluate
    model.load_state_dict(torch.load(f'models/model-{fold}.pt'))
    model.eval()
    preds = []
    targs = []
    for i, batch in enumerate(eval_loader):
        blocks = batch[-1]
        x = blocks[0].ndata['feat']
        with torch.no_grad():
            logits = model(blocks, x)

            labels = blocks[-1].dstdata['label']['article']
            
        preds.append(logits.argmax(-1).cpu().numpy())
        targs.append(labels.cpu().numpy())
    preds = np.concatenate(preds)
    targs = np.concatenate(targs)
    eval_results = {get_name(f):f(y_pred=preds, y_true=targs) for f in metrics}
    print("Final evaluation results:")
    for k,v in eval_results.items():
        print(f"{k:<16}{v:.4f}")
    
    wandb.log(eval_results)
    wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                            y_true=targs, preds=preds,
                            class_names=["Fake", "Real"])})

    return {
        'train_loss':train_loss,
        'train_acc':train_acc,
        'valid_loss':valid_loss,
        'valid_acc':valid_acc
    }

In [53]:
params = {
    "n_epochs":200,
    'bs': 32,
    'lr':4e-3,
    "seed":124,
    "lr_sched_factor":0.5,
    "patience":40
}

labels = graph.ndata['label']['article']

In [54]:
if not os.path.exists('models'):
    os.mkdir('models')

GROUP = "sbert-fulltext-gcn-B32P40updatedE200"
for fold_id, (train_idx, valid_idx) in enumerate(skf.split(labels, labels)):
    ipd.clear_output()
    with wandb.init(entity="saloniteam", project="nofolds", group=GROUP, name=f"{GROUP}-fold-{fold_id}") as run:
        log = train(fold_id, train_idx, valid_idx, params)
    break

wandb: Currently logged in as: saloni (saloniteam). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.7/dist-packages/dgl/dataloading/dataloader.py:859: DGLWarning: Dataloader CPU affinity opt is not enabled, consider switching it on (see enable_cpu_affinity() or CPU best practices for DGL [https://docs.dgl.ai/tutorials/cpu/cpu_best_practises.html])
  dgl_warning(f'Dataloader CPU affinity opt is not enabled, consider switching it on '
/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


  1: Train loss 0.5060, acc 0.7731%; validation loss 0.6865, acc 0.7789%
  2: Train loss 0.4739, acc 0.7969%; validation loss 0.4787, acc 0.7952%
  3: Train loss 0.4441, acc 0.8028%; validation loss 0.4749, acc 0.7752%
  4: Train loss 0.4662, acc 0.8067%; validation loss 0.5470, acc 0.7764%
  5: Train loss 0.4817, acc 0.7986%; validation loss 0.4977, acc 0.8197%
  6: Train loss 0.4396, acc 0.8104%; validation loss 0.5795, acc 0.7679%
  7: Train loss 0.4292, acc 0.8091%; validation loss 0.5093, acc 0.7804%
  8: Train loss 0.4306, acc 0.8108%; validation loss 0.4886, acc 0.8140%
  9: Train loss 0.4270, acc 0.8134%; validation loss 0.4838, acc 0.8057%
 10: Train loss 0.4291, acc 0.8131%; validation loss 0.4958, acc 0.7664%
 11: Train loss 0.4238, acc 0.8155%; validation loss 0.5116, acc 0.7922%
 12: Train loss 0.4256, acc 0.8111%; validation loss 0.5124, acc 0.8425%
 13: Train loss 0.4338, acc 0.8156%; validation loss 0.5411, acc 0.8435%
 14: Train loss 0.8403, acc 0.8068%; validation los

accuracy,▁
f1,▁
precision,▁
recall,▁
train_acc,▇▁█▁▃▆▅▆▆▃▆▆▆▅▆▇▆▇▇▄▃▆▅▃▅▆▃▅▆▆▆▇▆▆▄▇▆▇▆▆
train_loss,▃▆▃▇▄▃▄▄▄▄▂▄▂▇▅▂▇▄▅▃█▄▅▅▄▂▅▅▂▅▃▂▃▁▃▄▅▂▅▃
valid_acc,███████████▄▄███▁██▄▄▄▁▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄
valid_loss,▁▂▁▇▅▇▇█▃▄▅▅▄▄▄▃▃▄▃▃▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
accuracy,0.84352
f1,0.90037
precision,0.8861


In [52]:
model = GNN(graph, 128, n_layers, "article")

model.load_state_dict(torch.load(f'models/model-{fold_id}.pt'))

<All keys matched successfully>